### Import Libraries and Run Beautiful Soup

In [1]:
# Import Libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# NBA season we will be analyzing
year = 2019
# URL page we will scraping (see image above)
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

### Get the Column Headers

In [3]:
# use findALL() to get the column headers
soup.findAll('tr', limit=2)
# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
headers

['Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']

### Create the Pandas DataFrame

In [4]:
# avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [5]:
# Create the dataframe
stats = pd.DataFrame(player_stats, columns = headers)
stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,Tyler Zeller,C,29,MEM,4,1,20.5,4.0,7.0,.571,...,.778,2.3,2.3,4.5,0.8,0.3,0.8,1.0,4.0,11.5
730,Ante Žižić,C,22,CLE,59,25,18.3,3.1,5.6,.553,...,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8
731,Ivica Zubac,C,21,TOT,59,37,17.6,3.6,6.4,.559,...,.802,1.9,4.2,6.1,1.1,0.2,0.9,1.2,2.3,8.9
732,Ivica Zubac,C,21,LAL,33,12,15.6,3.4,5.8,.580,...,.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5


### Make the teams

In [6]:
# Find all the teams in the NBA
teams_prime = stats["Tm"]
teams = []
for team in teams_prime:
    if team not in teams:
        teams.append(team)
        
# TOT is not an NBA team.   Delete it.
del teams[13]
# delete the "nones"
del teams[17]
teams


['OKC',
 'PHO',
 'ATL',
 'MIA',
 'CLE',
 'DEN',
 'SAS',
 'CHI',
 'UTA',
 'BRK',
 'NYK',
 'POR',
 'MEM',
 'IND',
 'MIL',
 'DAL',
 'HOU',
 'TOR',
 'WAS',
 'ORL',
 'CHO',
 'SAC',
 'LAL',
 'MIN',
 'BOS',
 'GSW',
 'NOP',
 'LAC',
 'PHI',
 'DET']

In [7]:
# Create a team-specific data frame with the information we want
team = teams[0]
team_stats = stats.loc[(stats["Tm"] == team), ["Player", "Tm", "G", "PTS"]]
#Save a csv file 
team_stats.to_csv('../csv-files/NBA_Data_Scrape_{}_{}.csv'.format(team, year),index=False)

team_stats

,Player,Tm,G,PTS
0,Álex Abrines,OKC,31,5.3
3,Steven Adams,OKC,80,13.9
105,Deonte Burton,OKC,32,2.6
170,Tyler Davis,OKC,1,0.0
183,Hamidou Diallo,OKC,51,3.7
215,Jawun Evans,OKC,1,0.0
224,Raymond Felton,OKC,33,4.3
225,Terrance Ferguson,OKC,74,6.9
249,Paul George,OKC,77,28.0
260,Jerami Grant,OKC,80,13.6
